<a href="https://colab.research.google.com/github/Davidospina1000/Data-Integration/blob/main/5_Integraci%C3%B3nDin%C3%A1mica_Clase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0. Se procede con la carga e instalación de las librerias de trabajo

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


1. Se procede con la carga de lso archivos de trabajo

In [ ]:
nxl='/content/drive/MyDrive/2. IntegraciónDatos(01-2025)/6. NBA_Data.xlsx'
XDB=pd.read_excel(nxl,sheet_name=0)  #Base de datos observada
XDB.head(10)

XDB=XDB.iloc[:,1:6]
XDB.dropna()
XDB.head(10)

2. Se procede con el proceso de clusterización Estática (K-Means)

In [ ]:
np.random.seed(42)

#1. Se deben crear las semillas iniciales
XDBM=np.max(XDB,axis=0)
XDBm=np.min(XDB,axis=0)

XC=np.zeros((5,5))

for i in range(5): #Numéro de Montoncitos
  for j in range(5): #Cada Montoncito va a tener 5 variables
    XC[i,j]=np.random.uniform(XDBm[j],XDBM[j])

print(XC.astype(int))

XDB=np.array(XDB)
XCd=np.zeros((5,5))   #Los clusters despues
ND=np.zeros((5,1))

#2. Se procede con la clasificación de los datos por montoncito
for k in range(len(XDB)):
  d=np.sum(np.abs(XC[:,:]-XDB[k,:]),axis=1)  #Distancia de un registro a un Cluster
  fila=np.argmin(d)   #Número de cluster al que pertenece un dato

  XCd[fila,]=XCd[fila,]+XDB[k,:]  #Voy sumando la información de las variables a los clusters
  ND[fila,]=ND[fila,]+1  #Cuento el número de datos que quedaron por cluster

print("El número de datos por cluster es:\n",ND)

XCd=XCd/ND #Ahora dividimos todos las variables por le número de datos del cluster

dfxco=pd.DataFrame((XCd.astype(float)))
dfxco.columns=['Salary','Age','Height','Weigth','games_played']
dfxco.head(5)

#. Se procede a mostrar la correlación entrce las variables
dfcor=pd.DataFrame((XDB)).corr()
print("La correlación entre las variables es:\n",dfcor)

,Salary,Age,Height,Weigth,games_played
0,9.541074e+06,28.920000,79.140000,223.080000,589.720000
1,3.337586e+06,30.333333,78.904762,225.214286,558.214286
2,1.020454e+06,30.741935,79.096774,221.919355,546.580645
3,5.573482e+06,29.955556,78.911111,221.377778,565.000000
4,1.669724e+07,28.140000,79.820000,229.320000,572.300000


##**Integración Dinámica**

1. Se procede con la carga de los datos externos

In [ ]:
#1. Se carga la base de datos
nxle='/content/drive/MyDrive/2. IntegraciónDatos(01-2025)/6. NBA_Data.xlsx'
XDBe=pd.read_excel(nxle,sheet_name=1)
XDBe=np.array(XDBe.iloc[:,1:6].dropna())
#print("Los datos externos son:\n",XDBe)

#2. Mantenemos el historial de los clusters
XCe=np.copy(XCd)  #Cluster Estáticos o Iniciales
NDe=np.zeros((5,1)) #Número de datos que se integraron por cluster
nf=np.zeros((len(XDBe),1))  #Saber la fila de integración o cluster de recepción

for k in range(len(XDBe)):
  de=np.sqrt(np.sum((XCe[:,:]-XDBe[k,:])**2,axis=1))  #Distancia de un datos externo a los clusters internos
  fila=np.argmin(de)   #Número de cluster al que pertenece un dato
  nf[k,]=fila
  XCe[fila,]=(XCe[fila,]+XDBe[k,:])/2  #Voy sumando la información de las variables a los clusters

  NDe[fila,]=NDe[fila,]+1  #Cuento el número de datos que quedaron por cluster

print("El número de datos externos integrados por cluster es:\n",NDe)
print("La estructura de los cluster antes es:\n",pd.DataFrame(XCd))
print("La estructura de los cluster despues es:\n",pd.DataFrame(XCe))

#Los cambios porcentuales son:
print("los cambios porcentuales fueron:\n",(XCd-XCe)/XCd)

#El informe de integración es el siguiente:
dfinf=pd.DataFrame(np.column_stack((nf,XDBe)))
dfinf.columns=['Cluster','Salary','Age','Height','Weigth','games_played']
dfinf.head(10)

El número de datos externos integrados por cluster es:
 [[  0.]
 [ 46.]
 [133.]
 [ 15.]
 [  2.]]
La estructura de los cluster antes es:
               0          1          2           3           4
0  9.541074e+06  28.920000  79.140000  223.080000  589.720000
1  3.337586e+06  30.333333  78.904762  225.214286  558.214286
2  1.020454e+06  30.741935  79.096774  221.919355  546.580645
3  5.573482e+06  29.955556  78.911111  221.377778  565.000000
4  1.669724e+07  28.140000  79.820000  229.320000  572.300000
La estructura de los cluster despues es:
               0          1          2           3           4
0  9.541074e+06  28.920000  79.140000  223.080000  589.720000
1  2.998603e+06  19.888727  78.533124  215.260867   72.801505
2  9.240015e+05  21.548588  79.995629  199.007493   18.998757
3  5.239705e+06  20.271056  79.947019  220.888378   87.017883
4  1.508953e+07  25.535000  79.705000  231.830000  356.825000
los cambios porcentuales fueron:
 [[ 0.          0.          0.          0.  

,Cluster,Salary,Age,Height,Weigth,games_played
0,2.0,981348.0,25.0,79.0,240.0,219.0
1,1.0,3873398.0,23.0,80.0,225.0,307.0
2,1.0,3533333.0,25.0,78.0,175.0,254.0
3,3.0,5694674.0,22.0,77.0,207.0,247.0
4,3.0,6796117.0,25.0,78.0,235.0,311.0
5,2.0,947276.0,24.0,76.0,195.0,84.0
6,3.0,7070730.0,23.0,82.0,253.0,260.0
7,1.0,3272091.0,22.0,83.0,279.0,304.0
8,2.0,2008748.0,26.0,83.0,265.0,170.0
9,2.0,2288205.0,23.0,79.0,205.0,251.0


##**Análisis de Resultados**

En la integración de los datos se puede observar que el cluster que tuvo la menor integración o una integración nula fue el cluster 0, es por esto que los valores asociados a los cluster tuvieron una variación nula tal como se puede observar en la matriz de variaciones porcentuales. Se destaca el cluster 2, el cual tuvo la mayor cantidad de datos de integración, sin embargo, esta integración tuvo variaciones por encima del 80% en promedio con respecto a la variable games_played. Es por esto que se requiere establecer controles de integración con el fin de que los datos no tengan variaciones tan fuertes que pueden afectar la estructura de los clusters originales o asociados con la base de datos observada.